**AutoML OSS入門（8）**

# データ型ベースのディープラーニングAutoML OSS「Ludwig」

本ノートブックの紹介記事と併せてご覧ください。
- [＠IT連載 AutoML OSS入門（8）- 第8回「データ型ベースのディープラーニングAutoML OSS「Ludwig」」](https://atmarkit.itmedia.co.jp/ait/articles/2112/21/news003.html)

また、ノートブックの扱い方やタイタニックの生存予測データについては、連載記事の第1回も参照してください。
- [＠IT連載 AutoML OSS入門（１）- 第1回「機械学習モデル構築作業の煩雑さを解消する「AutoML」とは――歴史、動向、利用のメリットを整理する」](https://www.atmarkit.co.jp/ait/articles/2107/02/news006.html)

## livedoorニュースコーパスでAutoML
livedoorニュースコーパスを使って、Ludwigで文書分類を行ってみます。

学習にかなり時間がかかりますので、Colab上部の「ランタイム」タブから「ランタイムのタイプを変更」をクリックし、「ハードウェアアクセラレータ」を「GPU」に設定してください。

### セットアップ
Ludwigをインストールします。データの読み込みに使うライブラリであるPetastormも併せてインストールしておきます。<br>
ランタイムの再起動が求められた場合は、Colabの「ランタイム＞ランタイムを再起動」を実行してください。

In [ ]:
# 事前準備
!pip install ludwig
!pip install petastorm

In [ ]:
import ludwig
ludwig.__version__

'0.4'

### ライブラリーのインポート

In [ ]:
from ludwig.api import LudwigModel

### データのダウンロード
livedoorニュースコーパスをダウンロードし、整形を行います。

本コーパスはNHN Japan株式会社が運営するlivedoorニュースの日本語記事に対して9種類のカテゴリが付与されているものです。カテゴリにはスポーツやIT、家電などがあります。


In [ ]:
# livedoorニュースコーパスのダウンロード
!wget -N https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar zxvf ldcc-20140209.tar.gz

ストリーミング出力は最後の 5000 行に切り捨てられました。
text/smax/smax-6833065.txt
text/smax/smax-6833494.txt
text/smax/smax-6833579.txt
text/smax/smax-6834422.txt
text/smax/smax-6834562.txt
text/smax/smax-6834765.txt
text/smax/smax-6834990.txt
text/smax/smax-6835318.txt
text/smax/smax-6838218.txt
text/smax/smax-6838260.txt
text/smax/smax-6838332.txt
text/smax/smax-6838453.txt
text/smax/smax-6838514.txt
text/smax/smax-6838627.txt
text/smax/smax-6838702.txt
text/smax/smax-6842164.txt
text/smax/smax-6842219.txt
text/smax/smax-6842269.txt
text/smax/smax-6842402.txt
text/smax/smax-6842629.txt
text/smax/smax-6842882.txt
text/smax/smax-6843206.txt
text/smax/smax-6845003.txt
text/smax/smax-6847081.txt
text/smax/smax-6849018.txt
text/smax/smax-6850114.txt
text/smax/smax-6850391.txt
text/smax/smax-6850789.txt
text/smax/smax-6851590.txt
text/smax/smax-6851919.txt
text/smax/smax-6853063.txt
text/smax/smax-6853346.txt
text/smax/smax-6853751.txt
text/smax/smax-6854198.txt
text/smax/smax-6855347.txt
text/smax/smax-6855946.

In [ ]:
# ダウンロードしたニュース記事の形式を確認
!cat text/topic-news/topic-news-5986461.txt

http://news.livedoor.com/article/detail/5986461/
2011-11-01T14:19:00+0900
ツイッター上で週刊新潮にブチギレした橋下徹氏にネットでは賛否両論
10月31日、橋下徹前大阪府知事(42)がツイッター上で週刊新潮の取材を批判。これが現在ネットで話題となっている。

橋下氏はツイートで「週刊誌の記者ってそんなに偉いんでしょうか？」と、同誌の取材姿勢に疑問を呈し、「ある意味公人ですから、記者の情報も一緒に公開します」と前置きした上で、同誌の取材依頼書の内容を9回に分けてつぶやいた。

だが、そのつぶやきの中に、相手の電話番号なども含まれていたため、ネット掲示板やツイッターでは賛否両論が相次いでいる。

一連の橋下氏の対応に関し、ネットユーザーからは「もっとやれ」「これは橋下応援したい」と、橋下氏を擁護する声もあるが、全体的に見ると「電話番号晒しはまずい」「キレると何をするかわからない怖さがある」「橋下は責任転嫁の天才」などと橋下氏を批判する声も目立っている。

また、その他にも「週刊誌が一斉に同じ見出しで橋下叩き始めたことに作為的な悪意を感じるんだが、黒幕誰よ？」といったマスコミ批判の声も挙がっており、大阪府知事を退任し、今月27日には、大阪市長選を控える橋下氏——、その周辺はものものしい様相を呈している。

【関連情報】
・橋下氏、ツイッターで週刊新潮取材を大批判 - 日刊スポーツ新聞社

・橋下徹（さっきまで知事）が週刊新潮にブチギレ　twitterに喧嘩相手の電話番号を載せて反撃開始 - hogehoge速報

・橋下徹 - Twitter


In [ ]:
# ニュースコーパスをtsvファイルにまとめて保存
import os
text_dir = 'text/'
with open(text_dir + 'livedoor.tsv', 'w') as tsv_file:
    tsv_file.write('filename\tarticle\t')
    categories = [
                  f for f in os.listdir(text_dir)
                  if os.path.isdir(os.path.join(text_dir, f))]
    tsv_file.write('\t'.join(categories) + '\n')
    for (idx, category) in enumerate(categories):
        one_hot = ['0'] * 9
        one_hot[idx] = '1'
        for text_file in os.listdir(os.path.join(text_dir, category)):
            if text_file.startswith(category):
                with open(
                        os.path.join(text_dir, category, text_file), 'r') as f:
                    tsv_file.write(os.path.basename(f.name) + '\t')
                    [next(f) for _ in range(2)]
                    [tsv_file.write(line.replace('\n', ' ').replace(
                        '\t', ' ')) for line in f]
                tsv_file.write('\t'.join(one_hot) + '\n')

In [ ]:
# tsvファイルの確認
!head -10 ./text/livedoor.tsv

filename	article	topic-news	kaden-channel	livedoor-homme	sports-watch	dokujo-tsushin	peachy	smax	it-life-hack	movie-enter
topic-news-5955767.txt	チャン・グンソクの“乱入”に批判が殺到 チャン・グンソクの、授賞式における振舞いが韓国で非難されている。  17日に行われた韓国の大鐘賞映画祭で、受賞に感極まり涙を流している女優のキム・ハヌルのそばで、無関係なチャン・グンソクが乱入しVサインやウィンクをしたのだ。キム・ハヌルはグンソクとは次作の共演者という関係だが、グンソクとの共演作で受賞したわけではないので、韓国のネットユーザーからは「非常識」で「配慮が無い」と批判が殺到した。  この批判に対して、チャン・グンソクはTwitterで反応。「なぜウィンクをしたんだろう」と反省の呟きと同時に投稿した画像は何故か目のアップ。  思わずぎょっとするような画像を載せる“奇行”に、「非常識な振る舞いを反省していない」とさらなる批判を呼んでいる。  【関連情報】 ・チャン・グンソクの釈明のツイート https://twitter.com/#!/jksjapan/status/126089605031272448 https://twitter.com/#!/jksjapan/status/126926405173907456 https://twitter.com/#!/jksjapan/status/126925157515280384 ・장근석, 우는 김하늘 옆에서 윙크 논란에 "왜 했을까?" 해명（마이데일리） http://www.mydaily.co.kr/news/read.html?newsid=201110182030491110 【関連記事】 ・大人気なチャン・グンソクのファンがまわりに一人もいない理由 http://news.livedoor.com/article/detail/5949379/ 1	0	0	0	0	0	0	0	0
topic-news-6007462.txt	iPhoneに知らない間に中華製課金アプリがダウンロード 7日、ネット掲示板上では、「iPhoneに6つものアプリが勝手にダウンロードされて

In [ ]:
# tsvファイルを読み込んで整形
import pandas as pd
df = pd.read_table('text/livedoor.tsv', index_col=0)
df = df.set_index('article').stack().reset_index()
df = df[df[0] == 1].drop(0, axis=1).rename(
    columns={'article': 'text', 'level_1': 'class'}).reset_index(drop=True)
df

,text,class
0,盗撮現場を配信!? 無料イベント中継のはずが衝撃的瞬間が発覚【話題】 イベントの様子をネット...,topic-news
1,センター試験終了！ 受験生ファンに眞鍋かをりが的確なアドバイスをしていた【話題】 寒い週末、...,topic-news
2,KDDIがiPhone向けアプリ「auお客さまサポート」 をリリース【話題】 KDDIがiP...,topic-news
3,フジテレビのものまね番組がやりすぎ？ 劇団ひとりとAKB48のブータン国王夫妻のものまねがひ...,topic-news
4,学生は要チェック！ アップルがMacを買うと1万円分のアプリが買えるキャンペーンをスタート ...,topic-news
...,...,...
6592,『アーティスト』のミシェル・アザナヴィシウスが監督賞を受賞 第84回アカデミー賞の授賞式、...,it-life-hack
6593,“チャラ男”藤森が悩殺ビキニ美女たちと“ハワウィ〜ガールズ”結成 今年9月に全米3大ネット...,it-life-hack
6594,テレビ放送禁止の「お蔵入り恐怖映像」をウェブで公開 【注意※お蔵入】恐怖・・・。ホラー映画...,it-life-hack
6595,【週末映画まとめ読み】“ダークナイト”キャットウーマンの強烈バストにノックアウト！ 今週1...,it-life-hack


In [ ]:
# 9:1の割合で学習データとテストデータに分割する
test_df = df.sample(n=df.shape[0]//10, random_state=42)
train_df = df.drop(test_df.index)
print('学習データ数: {} ({:.2f}%)'.format(
    train_df.shape[0], 100*train_df.shape[0]/df.shape[0]))
print('テストデータ数: {} ({:.2f}%)'.format(
    test_df.shape[0], 100*test_df.shape[0]/df.shape[0]))

学習データ数: 5938 (90.01%)
テストデータ数: 659 (9.99%)


### モデルの学習

In [ ]:
# 辞書型でconfigを記載
config = {
    # データ分割の比率を設定する
    'preprocessing': {'split_probabilities': [0.9, 0.1, 0]},
    # 説明変数を設定する
    'input_features': [{'name': 'text',
                        'type': 'text',
                        'level': 'word'}],
    # 目的変数を設定する
    'output_features': [{'name': 'class',
                         'type': 'category'}]
          }

In [ ]:
# 学習
import logging
model = LudwigModel(config, logging_level=logging.INFO)
train_stats = model.train(train_df)

Experiment name: api_experiment
Model name: run
Output directory: results/api_experiment_run_2


ludwig_version: '0.4'
command: ('/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py -f '
 '/root/.local/share/jupyter/runtime/kernel-6e0cc582-9a25-4be5-a1f9-1f6325de4fe9.json')
random_seed: 42
data_format: "<class 'pandas.core.frame.DataFrame'>"
config: {   'combiner': {'type': 'concat'},
    'input_features': [   {   'column': 'text',
                              'encoder': 'parallel_cnn',
                              'level': 'word',
                              'name': 'text',
                              'proc_column': 'text_mZFLky',
                              'tied': None,
                              'type': 'text'}],
    'output_features': [   {   'column': 'class',
                               'dependencies': [],
                               'loss': {   'class_similarities_temperature': 0,
                                           'class_weights': 1,
         

### テストデータの予測
学習したモデルを用いてテストデータの予測を行います。

In [ ]:
# 予測
predictions = model.predict(test_df)[0]
result = pd.concat(
    [test_df.reset_index(drop=True),
     predictions[['class_predictions', 'class_probability']]], axis=1)
result

Prediction: 100%|██████████| 6/6 [00:00<00:00, 10.40it/s]


,text,class,class_predictions,class_probability
0,宮本恒靖氏、U-23日本代表に「リスクを冒さないとダメ」 11日、国立競技場では、ロンドン五...,livedoor-homme,livedoor-homme,0.997487
1,ヨンア「タンタンのように勇気のある男の子がすごく好き」 12月1日より公開となる、スティー...,it-life-hack,it-life-hack,0.998454
2,世界陸上期待の逸材＝Ｑちゃん似の中里麗美とは？ 韓国のテグを舞台に開幕する世界陸上競技選手権...,livedoor-homme,livedoor-homme,0.488105
3,お腹のハミ肉撃退エクサ／番号でわかる貯まる口座とは−【みんなのお気に入り登録記事】ランキング...,dokujo-tsushin,dokujo-tsushin,0.999385
4,今度は大丈夫？ iPhoneで24時間無料のWi-Fi！「LAWSON Wi-Fiサービス」...,smax,smax,0.999854
...,...,...,...,...
654,気をつけて！ スマートフォン、海外パケット定額プランのつもりが約100万円！のナゾ【話題】 ...,topic-news,topic-news,0.999995
655,冬の女子会は特製「薬膳火鍋」に決定！ 美味しく食べてポッカポカ 本格的な寒さを感じるこの季節...,dokujo-tsushin,sports-watch,0.793985
656,「尊敬できる上司がいません」 - 辛口説教部屋 vol.29 「3年で転職は早すぎる？」「将...,kaden-channel,kaden-channel,0.999888
657,ソニー、ＢＡ・ドライバーユニット搭載の密閉型インナーイヤーヘッドホンを発売【売れ筋チェック】...,topic-news,topic-news,0.999709


In [ ]:
# テストデータに対する正答率
sum(result['class'] == result['class_predictions']) / result.shape[0]

0.9271623672230652

In [ ]:
# 誤って分類した結果
result[result['class'] != result['class_predictions']]

,text,class,class_predictions,class_probability
9,スマホ初心者女子大生がドキドキ、パーソナルクラウド初体験 パーソナルクラウドサービスの「OC...,smax,dokujo-tsushin,0.963353
28,感謝を込めて贈りたい！歴女とっておきの”幕末ショコラ” NHK大河ドラマ「龍馬伝」の相乗効...,dokujo-tsushin,sports-watch,0.705843
52,選ぶのはあなた！5名の“ハッピー”な女性達がベッキーとの対面をかけてバトル 2008年12...,dokujo-tsushin,it-life-hack,0.675975
62,波乱の恋のドキドキ映画で恋のボルテージを上げる！ ロマンチックなクリスマス・・・を過ごした...,dokujo-tsushin,it-life-hack,0.711014
63,［第1回］オトコのホンネ座談会“独女と非独女は何か違うの？” 8月2日、独女通信がブログメデ...,kaden-channel,sports-watch,0.737260
64,奇跡の花が販売再開、青いバラの伝説とは？ 誰にも咲かせることのできない奇跡の花、青いバラ——...,sports-watch,dokujo-tsushin,0.350395
144,コピーライター50年の歴史を振り返る「コピーパレード展」開催 東京コピーライターズクラブ（T...,kaden-channel,livedoor-homme,0.809561
145,「麻雀必勝法か」亀田興毅の対戦相手にドッチラケの声 ＷＢＡ世界バンタム級王者・亀田興毅の４...,livedoor-homme,sports-watch,0.773753
153,新生活の第一歩を“いつもの音楽”で彩る 新しい生活に胸を躍らせる春。 厚生労働省の統計による...,kaden-channel,dokujo-tsushin,0.544153
154,イチゴ（1月15日）の日に限定コラボスイーツ誕生 14日、東京・自由が丘スイーツフォレスト...,dokujo-tsushin,kaden-channel,0.441971


以上で、livedoorニュースコーパスを使ったLudwigの紹介は終わりです。